# Split_5a  


In [1]:
import pickle
import pandas as pd 
import numpy  as np 
import papermill as pm
from pathlib import Path

In [2]:
import Notebook_5_helper_functions as Nb_5

In [3]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

HowManyBack = 1

Nb1= 'b'
Nb3='b'

data_dir_output = "/Users/gilmandelbaum/Desktop/output/"

Tags = ['Reward_NoReward_tag', 'ENL_tag', 'Cue_to_Sel_tag', "Cue_to_Sel_fromPrevTrial_tag","firstENLp_tag"]  
Timing = [600,200,15,15,20]
Start_or_end = ['start','end','start','start','start']

seq_str = "0a1b2a3b4ab5a6abc10a"

In [4]:
# Parameters
data_dir_input = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/0.data_Sets/1.lateral_medial_6/"
)
data_dir_output = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis_1/"
)
drop2blocks = "yes"
drop2lastblocks = "yes"
seq_str = "0a1b2a3l4abcd5a6a7a"
HowManyBack = 1
Nb0 = "a"
Nb1 = "b"
Nb2 = "a"
Nb3 = "l"
Nb4 = ["a", "b", "c", "d"]
Nb5 = "a"
Nb6 = "a"
Nb7 = "a"
Tags = [
    "Reward_NoReward_tag",
    "ENL_tag",
    "Cue_to_Sel_tag",
    "Cue_to_Sel_fromPrevTrial_tag",
]
Timing = [600, 200, 15, 15]
Start_or_end = ["start", "end", "start", "start"]
y_axis_list = ["df_f", "z_score", "z_score_sliding_window"]
mouse = "T169"
data_day = "2019_02_25__T169"
date = "2019_02_25"
photo_day = "T169-145248"


In [5]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_1_"+Nb1[-1]+'.pickle'
my_path = root / d 
# open a file, where you stored the pickled data
fileToOpen = open(my_path, 'rb')
# dump information to that file
behavior_data_trial_RIC_LIC_combinations_n = pickle.load(fileToOpen)

In [6]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_3_"+Nb3+'_seq'+seq_str[seq_str.index("2"):seq_str.index("4")]+'.pickle'
my_path = root / d 
# open a file, where you stored the pickled data
fileToOpen = open(my_path, 'rb')
# dump information to that file
processed_data = pickle.load(fileToOpen)

In [7]:
aligned_behav_photo = Nb_5.combine_processed_data_and_tags (Tags,processed_data,root)

In [8]:
aligned_behav_photo = Nb_5.tag_Pre_Post_Periods(aligned_behav_photo, 200) 

In [9]:
BehPhoto_RL_IC_tt_periods = Nb_5.behavPhoto_split(aligned_behav_photo, behavior_data_trial_RIC_LIC_combinations_n)

# Drop columns so the data frames only have photometry data that corresponds to left and right side. also name change occurs here to make names shorter



In [10]:
BehPhoto_RL_IC_tt_periods=Nb_5.drop_columns(BehPhoto_RL_IC_tt_periods)

In [ ]:
PhotoData_perTrial_channels = Nb_5.get_photodata_perTrial_df_firstStep(BehPhoto_RL_IC_tt_periods)

In [ ]:
for i in range(len(Tags)):
    PhotoData_perTrial_channels = Nb_5.get_specificTime_fromPeriod(PhotoData_perTrial_channels, 
                                                                   Tags[i], Timing[i], Start_or_end[i])

In [ ]:
list_of_tags= pd.DataFrame(PhotoData_perTrial_channels[0][0][3].keys(),columns=["list_of_tags"])
list_of_tags.to_csv(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back"+"/"+seq_str[:seq_str.index('6')]+"_list_of_tags.csv")

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+'Notebook_5_a'+'_seq'+seq_str[:seq_str.index('6')]+'.pickle'
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump((PhotoData_perTrial_channels),my_file)